<a href="https://colab.research.google.com/github/YanivAmir/stock-predicition-CNN/blob/main/loading_uniting_and_downsampling_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import yfinance as yf
# import yahoo_fin.stock_info as si
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import random
# import mplfinance as mpf
from datetime import datetime
from collections import Counter
import pandas as pd #pip install pandas==1.3.0
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import logging
import re
import seaborn as sns
from dateutil.relativedelta import relativedelta
from copy import deepcopy
import warnings
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader,TensorDataset
from torchsummary import summary
import torch.cuda.amp as amp
from torch.cuda.amp import autocast, GradScaler

warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive')
savePath ='/content/drive/MyDrive/myStocksSept2024/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading and uniting the separate data file of >10% and below <-10%
#### candlesticks and market cap are uniting into 2-channel images

In [ ]:
def loadSingleExtremeDataFile(path, name,strt,end):
  data = np.load(path+name+'_asList_'+str(strt)+'-'+str(end)+'_24Sept2024.npz')
  data_array = np.array([data[key] for key in data])
  return(torch.from_numpy(data_array).half())

def loadExtremeTargetsData(path,strts,pool=1):
  images_list,labels_list = [],[]
  for strt,end in zip(strts[:-1],strts[1:]):
    print(strt)
    stockImgs = F.avg_pool2d(loadSingleExtremeDataFile(path,'StockImgs',strt,end),pool)
    capImgs = F.avg_pool2d(loadSingleExtremeDataFile(path,'CapImgs',strt,end),pool)
    print(f'Plot sizes after avg pooling : {stockImgs.shape}')
    images = torch.cat((stockImgs.unsqueeze(1), capImgs.unsqueeze(1)), dim=1)
    labels = loadSingleExtremeDataFile(path,'Targets',strt,end)
    images_list.append(images)
    labels_list.append(labels)
  images = torch.cat((images_list), dim=0)
  labels = torch.cat((labels_list), dim=0)
  return(images,labels)


two seperate date file are generated:

In [ ]:
extremeTargetIndx = [0,118,271,456,638,811,841,1000,1174,1342,1526,1594,1851,2000]
images,targets= loadExtremeTargetsData(savePath,extremeTargetIndx,pool=3)
print(f'Images sizes after uniting data : {images.shape}')
print(f'Targets sizes after uniting data : {targets.shape}')
torch.save(images, save+ath+'images_stocks0-2000_images0-54383_pool3_asHalf.pt')
torch.save(targets,savePath+'labels_stocks0-2000_images0-54383_pool3_asHalf.pt')

extremeTargetIndx = [2000,2291,2500,2678,2738,3000,3167,3315]
images,targets= loadExtremeTargetsData(savePath,extremeTargetIndx,pool=3)
print(f'Images sizes after uniting data : {images.shape}')
print(f'Targets sizes after uniting data : {targets.shape}')
torch.save(images, savePath+'images_stocks2000-3315_images54383-77210_pool3_asHalf.pt')
torch.save(targets,savePath+'labels_stocks2000-3315_images54383-77210_pool3_asHalf.pt')


united into a single data file at float16

In [ ]:
images0 = torch.load(savePath+'images_stocks0-2000_images0-54383_pool3.pt').half()
targets0 = torch.load(savePath+'labels_stocks0-2000_images0-54383_pool3.pt').half()
images1 = torch.load(savePath+'images_stocks2000-3315_images54383-77210_pool3.pt').half()
targets1 = torch.load(savePath+'labels_stocks2000-3315_images54383-77210_pool3.pt').half()
images0 = torch.cat((images0, images1), dim=0)
targets0 = torch.cat((targets0, targets1), dim=0)
print(f'Images sizes after uniting data : {images.shape}')
print(f'Targets sizes after uniting data : {targets.shape}')

torch.save(targets0, savePath+'labels_stocks0-3315_images0-77210_pool3_asHalf.pt')
torch.save(images0, savePath+'images_stocks0-3315_images0-77210_pool3_asHalf.pt')

### Loading and uniting the separate data file of >5% and below <5%
#### candlesticks and market cap are uniting into 2-channel images

In [ ]:
def loadSingleZeroTargetsDataFile(path, name,strt,end):
  data = np.load(path+name+'_asList_'+str(strt)+'-'+str(end)+'_24Sept2024_targetsBelow5.npz')
  data_array = np.array([data[key] for key in data])
  return(torch.from_numpy(data_array).half())

def loadZeroTargetsData(path,startindices,endindiceses,pool=1):
  images_list,labels_list = [],[]
  for strt,end_indices in zip(startindices,endindiceses):
    for end in end_indices:
      print(strt,' ',end)
      stockImgs = F.avg_pool2d(loadSingleZeroTargetsDataFile(path,'StockImgs',strt,end),pool)
      capImgs = F.avg_pool2d(loadSingleZeroTargetsDataFile(path,'CapImgs',strt,end),pool)
      images = torch.cat((stockImgs.unsqueeze(1), capImgs.unsqueeze(1)), dim=1)
      labels = loadSingleZeroTargetsDataFile(path,'Targets',strt,end)
      print(images.shape)
      images_list.append(images)
      labels_list.append(labels)
      print(len(labels_list))
  images = torch.cat((images_list), dim=0)
  labels = torch.cat((labels_list), dim=0)
  return(images,labels)

two seperate date file are generated as float16:


In [ ]:
zerosStartIndices_1 = [0,195,569,1362,1500,1951]
zerosEndIndiceses_1= [
  [95,195],
  [255,317,384,448,513,569],
  [636,697,763,828,895,963,1034,1100,1163,1229,1295,1362],
  [1428,1491,1499],
  [1556,1628,1679,1734,1789,1843,1897,1951],
  [1999]]
zerosStartIndices_2 = [2000,2637,2797,2850,3000]
zerosEndIndiceses_2= [
  [2058,2112,2163,2213,2264,2319,2374,2426,2476,2530,2584,2637],
  [2686,2739,2797],
  [2849],
  [2903,2956,2999],
  [3057,3215,3269,3315,3269,3315]
  ]

images,targets= loadZeroTargetsData(savePath,zerosStartIndices_1,zerosEndIndiceses_1,pool=3)
torch.save(targets, savePath+'labels_stocks0-2000_targetsBelow5_pool3_asHalf.pt')
torch.save(images, savePath+'images_stocks0-2000_targetsBelow5_pool3_asHalf.pt')
print(f'Images sizes after uniting data : {images.shape}')
print(f'Targets sizes after uniting data : {targets.shape}')
images,targets = [],[]
images,targets= loadZeroTargetsData(savePath,zerosStartIndices_2,zerosEndIndiceses_2,pool=3)
torch.save(targets, savePath+'labels_stocks2000-3315_targetsBelow5_pool3_asHalf.pt')
torch.save(images, savePath+'images_stocks2000-3315_targetsBelow5_pool3_asHalf.pt')
print(f'Images sizes after uniting data : {images.shape}')
print(f'Targets sizes after uniting data : {targets.shape}')

to generate a more balanced dataset or equally numbered labels:

-0.3 < targets < -0.1
-0.05 < targets < 0.05
0.1 < targets < 0.3

In [ ]:
images0 = torch.load(savePath+'images_stocks0-2000_images0-54383_pool3.pt').half()[::2]
targets0 = torch.load(savePath+'labels_stocks0-2000_images0-54383_pool3.pt').half()[::2]
images1 = torch.load(savePath+'images_stocks2000-3315_images54383-77210_pool3.pt').half()[::2]
targets1 = torch.load(savePath+'labels_stocks2000-3315_images54383-77210_pool3.pt').half()[::2]
images0 = torch.cat((images0, images1), dim=0)
targets0 = torch.cat((targets0, targets1), dim=0)
print(f'Images sizes after uniting data : {images0.shape}')
print(f'Targets sizes after uniting data : {targets0.shape}')
images1=[]
images2 = torch.load(savePath+'images_stocks0-2000_targetsBelow5_pool3_asHalf.pt').half()[::8]
targets2 = torch.load(savePath+'labels_stocks0-2000_targetsBelow5_pool3_asHalf.pt').half()[::8]
images3 = torch.load(savePath+'images_stocks2000-3315_targetsBelow5_pool3_asHalf.pt').half()[::8]
targets3 = torch.load(savePath+'labels_stocks2000-3315_targetsBelow5_pool3_asHalf.pt').half()[::8]
images2 = torch.cat((images2, images3), dim=0)
targets2 = torch.cat((targets2, targets3), dim=0)
print(f'Images sizes after uniting data : {images2.shape}')
print(f'Targets sizes after uniting data : {targets2.shape}')
images3 = []
images2 = torch.cat((images0, images2), dim=0)
targets2 = torch.cat((targets0, targets2), dim=0)
images0=[]
print(f'Images sizes after uniting data : {images2.shape}')
print(f'Targets sizes after uniting data : {targets2.shape}')

torch.save(targets2, savePath+'labels_allStocks_midAndExtremes_morebalanced_pool3_asHalf.pt')
torch.save(images2, savePath+'images_allStocks_midAndExtremes_morebalanced_pool3_asHalf.pt')

In [ ]:
targets = torch.load(savePath+'labels_allStocks_midAndExtremes_morebalanced_pool3_asHalf.pt')
images = torch.load(savePath+'images_allStocks_midAndExtremes_morebalanced_pool3_asHalf.pt')
print(f'Images sizes after uniting data : {images.shape}')
print(f'Targets sizes after uniting data : {targets.shape}')

to generate a more equally volume reduced dataset:

In [ ]:
images0 = torch.load(savePath+'images_stocks0-2000_images0-54383_pool3.pt').half()[::3]
targets0 = torch.load(savePath+'labels_stocks0-2000_images0-54383_pool3.pt').half()[::3]
images1 = torch.load(savePath+'images_stocks2000-3315_images54383-77210_pool3.pt').half()[::3]
targets1 = torch.load(savePath+'labels_stocks2000-3315_images54383-77210_pool3.pt').half()[::3]
images0 = torch.cat((images0, images1), dim=0)
targets0 = torch.cat((targets0, targets1), dim=0)
print(f'Images sizes after uniting data : {images0.shape}')
print(f'Targets sizes after uniting data : {targets0.shape}')
images1=[]
images2 = torch.load(savePath+'images_stocks0-2000_targetsBelow5_pool3_asHalf.pt').half()[::3]
targets2 = torch.load(savePath+'labels_stocks0-2000_targetsBelow5_pool3_asHalf.pt').half()[::3]
images3 = torch.load(savePath+'images_stocks2000-3315_targetsBelow5_pool3_asHalf.pt').half()[::3]
targets3 = torch.load(savePath+'labels_stocks2000-3315_targetsBelow5_pool3_asHalf.pt').half()[::3]
images2 = torch.cat((images2, images3), dim=0)
targets2 = torch.cat((targets2, targets3), dim=0)
print(f'Images sizes after uniting data : {images2.shape}')
print(f'Targets sizes after uniting data : {targets2.shape}')
images3 = []
images2 = torch.cat((images0, images2), dim=0)
targets2 = torch.cat((targets0, targets2), dim=0)
images0=[]
print(f'Images sizes after uniting data : {images2.shape}')
print(f'Targets sizes after uniting data : {targets2.shape}')

torch.save(targets2, savePath+'labels_allStocks_midAndExtremes_every3rdImg_pool3_asHalf.pt')
torch.save(images2, savePath+'images_allStocks_midAndExtremes_every3rdImg_pool3_asHalf.pt')

In [ ]:
images = torch.load(savePath+'images_allStocks_midAndExtremes_every3rdImg_pool3_asHalf.pt')
targets = torch.load(savePath+'labels_allStocks_midAndExtremes_every3rdImg_pool3_asHalf.pt')
print(f'Images sizes after uniting data : {images.shape}')
print(f'Targets sizes after uniting data : {targets.shape}')